# Teil 4 Demo 2: Abfragen mit Tidyverse

## Bibliotheken & Konfiguration

In [ ]:
packages <- c("readr", "dplyr", "stringr", "tidyr")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-Datenanalyse-Visualisierung/master/Rohdaten/mimic-iii-demo/"

## Ausgewählte MIMIC III-Rohdaten laden

In [ ]:
mimic.patients.raw      <- read_csv(paste(base_url, "PATIENTS.csv", sep=""),
                                    col_types = cols(row_id = col_double(), subject_id = col_double(), gender = col_character(), dob = col_datetime(format = ""), dod = col_datetime(format = ""), dod_hosp = col_datetime(format = ""), dod_ssn = col_datetime(format = ""), expire_flag = col_double()))
mimic.admissions.raw    <- read_csv(paste(base_url, "ADMISSIONS.csv", sep=""),
                                    col_types = cols(  row_id = col_double(), subject_id = col_double(), hadm_id = col_double(), admittime = col_datetime(format = ""), dischtime = col_datetime(format = ""), deathtime = col_datetime(format = ""), admission_type = col_character(), admission_location = col_character(), discharge_location = col_character(), insurance = col_character(), language = col_character(), religion = col_character(), marital_status = col_character(), ethnicity = col_character(), edregtime = col_datetime(format = ""), edouttime = col_datetime(format = ""), diagnosis = col_character(), hospital_expire_flag = col_double(), has_chartevents_data = col_double()))
mimic.prescriptions.raw <- read_csv(paste(base_url, "PRESCRIPTIONS.csv", sep=""),
                                    col_types = cols(row_id = col_double(), subject_id = col_double(), hadm_id = col_double(), icustay_id = col_double(), startdate = col_datetime(format = ""), enddate = col_datetime(format = ""), drug_type = col_character(), drug = col_character(), drug_name_poe = col_character(), drug_name_generic = col_character(), formulary_drug_cd = col_character(), gsn = col_character(), ndc = col_character(), prod_strength = col_character(), dose_val_rx = col_character(), dose_unit_rx = col_character(), form_val_disp = col_character(), form_unit_disp = col_character(), route = col_character()))
#head(mimic.patients.raw)
#head(mimic.admissions.raw)
#head(mimic.prescriptions.raw)

## Alle Spalten eines Tibble abfragen

In [ ]:
head(mimic.patients.raw)

## Ausgewählte Spalten eines Tibble abfragen

In [ ]:
head(mimic.patients.raw %>% select(subject_id, gender))

## Ausgewählte Zeilen eines Tibble abfragen

In [ ]:
head(mimic.patients.raw %>% filter(gender == 'M'))

# Merke: bei Vergleichen muss das doppelte Gleichheitszeichen (==) verwendet werden!

## Ergebnis auf eindeutige Datensätze reduzieren (DISTINCT)

In [ ]:
mimic.patients.raw %>% 
    select(gender) %>% 
    distinct()

## Einfache Aggregation: Gesamtzahl der Datensätze eines Tibble abfragen

In [ ]:
mimic.patients.raw %>% summarize(n = n())

## Aggregation & Gruppierung: Anzahl nach Geschlecht ermitteln

In [ ]:
mimic.patients.raw %>% 
    group_by(gender) %>% 
    summarize(n = n(), .groups="keep")

# Merke: die Option .groups="keep" ist nötig, damit das Gruppierungsmerkmal in der Ausgabe übernommen wird!

## Zeilen nach der Aggregation filtern

In [ ]:
mimic.patients.raw %>% 
    group_by(gender) %>% 
    summarize(n = n(), .groups="keep") %>% 
    filter(n > 50)

## Tibbles per JOIN miteinander verknüpfen

In [ ]:
head(mimic.patients.raw %>% 
     inner_join(mimic.admissions.raw, by = "subject_id") %>% 
     select(subject_id, gender, diagnosis)
)

## Patient:innen mit mehr als einem Intensiv-Aufenthalt abfragen

In [ ]:
head(mimic.patients.raw %>% 
     inner_join(mimic.admissions.raw, by = "subject_id") %>% 
     group_by(subject_id) %>% 
     summarize(n = n(), .groups="keep") %>% 
     filter(n > 1)
)

## 2 Subsets von Verschreibungen für die Demo verschiedener Joins erzeugen

* Patient:innenen mit Hauptdiagnose Sepsis
* Patient:innen mit Gabe von Vancomycin (Reserve-Antibiotikum z.B. bei MRSA)

In [ ]:
demo.sepsis     <- mimic.admissions.raw %>% 
                        filter(str_detect(tolower(diagnosis), "sepsis")) %>% 
                        select(subject_id) %>%
                        distinct()
demo.vancomycin <- mimic.prescriptions.raw %>% 
                        filter(str_detect(tolower(drug), "vancomycin")) %>% 
                        select(subject_id) %>% 
                        distinct()

## Patient:innen abfragen, die sowohl eine Sepsis als Hauptdiagnose hatten als auch Vancomycin erhalten haben (INNER JOIN)

In [ ]:
demo.sepsis %>% 
    inner_join(demo.vancomycin, by = "subject_id", keep=TRUE, suffix = c("_sepsis", "_vanco"))

## Alle Patient:innen abfragen, die eine Sepsis als Hauptdiagnose hatten sowie die mit Vancomycingabe ergänzen (LEFT OUTER JOIN)

In [ ]:
demo.sepsis %>% 
    left_join(demo.vancomycin, by = "subject_id", keep=TRUE, suffix = c("_sepsis", "_vanco"))

## Alle Patient:innen abfragen, die eine Vancomycingabe hatten und die keine Sepsisdiagnose hatten (RIGHT OUTER JOIN)

In [ ]:
head(demo.sepsis %>% 
     right_join(demo.vancomycin, by = "subject_id", keep=TRUE, suffix = c("_sepsis", "_vanco")),
15)

## Tatsächliche Hauptdiagnosen für die Patient:innen mit Vancomycingabe ohne Sepsis ermitteln

In [ ]:
head(demo.vancomycin %>% 
     inner_join(mimic.admissions.raw, by = "subject_id") %>% 
     left_join(demo.sepsis, by = "subject_id", keep=TRUE, suffix=c("", "_sepsis")) %>% 
     filter(is.na(subject_id_sepsis)) %>% 
     group_by(diagnosis) %>% 
     summarize(n = n(), .groups="keep") %>% 
     arrange(desc(n)) 
, 20)